## Imports and functions

In [ ]:
!pip install regex

In [ ]:
import regex
import requests
import json
import socket
import re
import base64
import os

In [ ]:
ip = os.environ.get('pod_ip')
login_name = os.environ.get('pod_login')
my_pod_ip = ''

### Scanning functions

In [ ]:
def check_port(ip, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(0.1)
    try:
        s.connect((ip, port))
        #print(f"Server found at {ip}:{port}                ")
        return True
    except socket.error as e:
        #print(f"No server found at {ip}:{port}             ", end="\r")
        #print("\r", end="")
        return False


def scan_range(start_ip, end_ip, port, server_ips):
    start_ip_parts = [int(part) for part in start_ip.split(".")]
    end_ip_parts = [int(part) for part in end_ip.split(".")]

    current_ip_parts = start_ip_parts

    while current_ip_parts != end_ip_parts:
        current_ip = ".".join(str(part) for part in current_ip_parts)
        if check_port(current_ip, port):
            server_ips.append(current_ip)
        current_ip_parts[3] += 1
        for i in range(3, 0, -1):
            if current_ip_parts[i] == 256:
                current_ip_parts[i] = 0
                current_ip_parts[i-1] += 1

    if check_port(end_ip, port):
        server_ips.append(end_ip)

def check_jupyter_logins(server_ips):
    ips_logins = []
    for ip in server_ips:
        if ip==my_pod_ip: continue
        response = requests.get('http://' +  ip + ':8888/', timeout=1)
        login=regex.findall(r'rhods-notebooks\/\K(.*?)\/',response.content.decode("utf-8"))
        if len(login)>0:
            ips_logins.append((ip,login[0]))
    
    if len(ips_logins)>0:
        print("Here are the Pod IPs and the associated logins:")
        print("-----------------------------------------------")
        for element in ips_logins:
            print(element[0] + ' -> ' + element[1])
    else:
        print("No Jupyter pods found")

def print_tree_view(location):
    try:
        response = requests.get('http://' +  ip + ':8888/notebook/rhods-notebooks/' + login_name + '/api/contents' + location, timeout=10)
        print("Here are all the accessible files:")
        print("----------------------------------")
        contentData = response.json()['content']
        if isinstance(contentData, list):
            for item in contentData:
                if item['type'] != 'directory':
                    print(item['path'])
                else:
                    print_tree_view('/' + item['path'])
    except requests.ConnectTimeout:
        print('Server did not respond after 10 seconds, assuming connection is blocked')
        
def scan_pods():
    ip_mask = '.'.join(ip.split('.')[:3])
    start_ip = ip_mask+'.0'
    end_ip = ip_mask+'.255'
    port = 8888
    server_ips = []
    scan_range(start_ip, end_ip, port, server_ips)
    if len(server_ips)>0:
        print("Servers found at:")
        for _ in server_ips:
            if _ == my_pod_ip: print(_+' (my own server)')
            else: print(_)
    else:
        print("No servers found")
    return server_ips